In [1]:
filename = 'reu_text_10Jul.csv'

In [2]:
import pandas as pd
df = pd.read_csv(filename)
df.head()


,Run ID,Name,Source Type,Source Name,User,Status,corpus_name,corpus_type,embedding,fe_type,...,positive_label_in_total,precison/labels,precison/selected_labels,recall/labels,recall/selected_labels,selected_features_count,singleton_clusters_count,tnr/labels,tnr/selected_labels,unassigned_doc_percent
0,41dddabc9b7044c7a6cbce9d21b428e4,NaN,LOCAL,/Users/vdixit/SELF_READING/Projects/git/hierar...,vdixit,FINISHED,Reuters-text,long,sentbert,default,...,11.891,4.7879,0.7071,89.7638,100.0000,1342,20,77.33,36.81,88.108662
1,84727793606c42018e29ba227df34cff,NaN,LOCAL,/Users/vdixit/SELF_READING/Projects/git/hierar...,vdixit,FINISHED,Reuters-text,long,sentbert,default,...,21.527,12.1443,4.5822,93.8080,100.0000,390,20,78.08,52.73,78.472578
2,440f1cffbbb04272b9d3743a888ab56e,NaN,LOCAL,/Users/vdixit/SELF_READING/Projects/git/hierar...,vdixit,FINISHED,Reuters-text,long,sentbert,default,...,21.630,11.3444,4.2788,92.1311,100.0000,390,20,78.04,52.35,78.370067
3,ffb772ea50dd4fe0b3f1f1d83bf178eb,NaN,LOCAL,/Users/vdixit/SELF_READING/Projects/git/hierar...,vdixit,FINISHED,Reuters-text,long,sentbert,default,...,30.958,15.4924,8.7181,94.2396,99.0741,195,27,77.69,66.39,69.041517
4,f7b688333f1b4ae6b3c96a2252a77959,NaN,LOCAL,/Users/vdixit/SELF_READING/Projects/git/hierar...,vdixit,FINISHED,Reuters-text,long,sentbert,default,...,30.856,15.6901,8.4041,94.6512,99.0323,195,27,78.13,66.54,69.144029


In [3]:
input_columns = ['embedding', 'fe_type', 'max_features_frac',
       'max_overlap_percent',
       'num_embedding_passes',
       'with_negative_label']


In [4]:
df.isna().sum()

Run ID                                    0
Name                                     49
Source Type                               0
Source Name                               0
User                                      0
Status                                    0
corpus_name                               0
corpus_type                               0
embedding                                 0
fe_type                                   0
max_features_frac                         0
max_labels_per_doc                        0
max_overlap_percent                       0
neg_pairs/labels                          0
neg_pairs/selected_labels                 0
num_embedding_passes                     12
outlier_threshold                         0
pos_pairs/labels                          0
pos_pairs/selected_labels                 0
total_documents                           0
total_pairs/labels                        0
total_pairs/selected_labels               0
with_negative_label             

In [5]:
df = df.fillna('None')

In [6]:
feat_pred_cols = ['absent_features/labels', 'absent_features/selected_labels', 'assigned_doc_percent']
metrics_cols =  ['accuracy/labels','accuracy/selected_labels', 'fscore/labels', 'fscore/selected_labels','outlier_precision/labels', \
                 'outlier_precision/selected_labels', 'precison/labels', 'precison/selected_labels', 'recall/labels',\
                'recall/selected_labels', 'tnr/labels', 'tnr/selected_labels']

In [7]:
filter_cols = ['fe_type', 'num_embedding_passes', 'max_features_frac', 'max_overlap_percent']

In [8]:
def select_feats(output_f, op_function):
    temp_df = df.copy()
    filters_map = {}
    
    for f_col in filter_cols:
        best_out_val = op_function(df[output_f])
        best_f_val = df[df[output_f]==best_out_val][f_col].iloc[0]
       # print(f_col, df[f_col].unique())
        if f_col == 'num_embedding_passes':
            temp_df = temp_df[temp_df[f_col].isin([best_f_val, 'None'])]
       # print(f_col, temp_df.embedding.unique(), '\n')
        filters_map[f_col] =  {'best_val':best_f_val, 'best_out_val':best_out_val, 'df_size':len(temp_df)}
    
    return temp_df, filters_map


_, f_map =select_feats('absent_features/labels', min)
f_map


{'fe_type': {'best_val': 'default', 'best_out_val': 1.0251, 'df_size': 49},
 'num_embedding_passes': {'best_val': 4.0,
  'best_out_val': 1.0251,
  'df_size': 31},
 'max_features_frac': {'best_val': 'None',
  'best_out_val': 1.0251,
  'df_size': 31},
 'max_overlap_percent': {'best_val': '2',
  'best_out_val': 1.0251,
  'df_size': 31}}

In [9]:
for col in feat_pred_cols[2:] + metrics_cols:
    print(col)
    _,select_map = select_feats(col, max)
    for k, v in select_map.items():
        print(k, v)
    print('\n')

assigned_doc_percent
fe_type {'best_val': 'default', 'best_out_val': 34.905, 'df_size': 49}
num_embedding_passes {'best_val': 'None', 'best_out_val': 34.905, 'df_size': 12}
max_features_frac {'best_val': '0.1', 'best_out_val': 34.905, 'df_size': 12}
max_overlap_percent {'best_val': '3', 'best_out_val': 34.905, 'df_size': 12}


accuracy/labels
fe_type {'best_val': 'default', 'best_out_val': 82.9297, 'df_size': 49}
num_embedding_passes {'best_val': 'None', 'best_out_val': 82.9297, 'df_size': 12}
max_features_frac {'best_val': '0.1', 'best_out_val': 82.9297, 'df_size': 12}
max_overlap_percent {'best_val': '3', 'best_out_val': 82.9297, 'df_size': 12}


accuracy/selected_labels
fe_type {'best_val': 'default', 'best_out_val': 74.7253, 'df_size': 49}
num_embedding_passes {'best_val': 'None', 'best_out_val': 74.7253, 'df_size': 12}
max_features_frac {'best_val': '0.1', 'best_out_val': 74.7253, 'df_size': 12}
max_overlap_percent {'best_val': '3', 'best_out_val': 74.7253, 'df_size': 12}


fscore